In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense


In [11]:
# from google.colab import drive
# drive.mount('/content/drive')

data_dir = '../dataset/wiki/'

In [25]:
df = pd.read_csv(data_dir + 'shuffled_GPT_wiki_intro.csv', header=None)

/var/folders/h4/qv9ymlxn5v54y1vnffsnvmlr0000gn/T/ipykernel_35544/3007117991.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_dir + 'shuffled_GPT_wiki_intro.csv', header=None)


In [27]:

df = df[:50000]
print(df)

           0                                                  1       2
0         id                                               text  labels
1          0  Hope (born 2010) is a bottlenose dolphin who i...       1
2          1  The Invasion of Umuahia (December 20-December ...       0
3          2  Morgan Adokwei Brown is a lawyer and human rig...       1
4          3  Martine Warmann (formerly Martine George; born...       1
...      ...                                                ...     ...
49995  49994  Chessgames.com is an Internet chess community ...       0
49996  49995  Ahmed Sudi, also known as Sudi-yi (or Sudi-al-...       1
49997  49996  The Budapest offensive was the general attack ...       1
49998  49997  Rochelle Ballard (born February 13, 1971) is a...       1
49999  49998  West Ogwell is a village and former parish in ...       1

[50000 rows x 3 columns]


In [29]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# 文本预处理
max_words = 10000  # 仅使用最常见的10000个词
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data.iloc[:, 1])  # 第二列是文本

X_train = tokenizer.texts_to_sequences(train_data.iloc[:, 1])
X_test = tokenizer.texts_to_sequences(test_data.iloc[:, 1])

max_sequence_length = max(len(seq) for seq in X_train)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

# 标签预处理
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data.iloc[:, 2])  # 第三列是标签
y_test = label_encoder.transform(test_data.iloc[:, 2])

# 建立GRU模型
embedding_dim = 100
hidden_units = 64

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(GRU(units=hidden_units))
model.add(Dense(1, activation='sigmoid'))

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, epochs=1, batch_size=32, validation_split=0.2)

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

313/313 [==============================] - 6s 20ms/step - loss: 0.1884 - accuracy: 0.9267
Test Accuracy: 92.67%
